In [1]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [2]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv").mapPartitions(lambda x: csv.reader(x))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(line[TEXT],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] )).take(5)

print trainRDD


[('492346', [1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2]), ('343811', [0, 5, 1, 2, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 5, 1, 0, 0, 1, 5, 0, 0]), ('347257', [0, 5, 3, 1, 0, 3, 0, 1, 3, 1, 7, 3, 2, 1, 3, 2, 2, 3, 3, 0, 2, 1, 0, 3, 5, 1, 0, 0, 1, 5, 7, 1]), ('225529', [1, 0, 1, 0, 4, 1, 10, 0, 3, 0, 0, 0, 3, 0, 3, 2, 1, 3, 3, 4, 0, 0, 4, 1, 0, 0, 0, 2, 0, 0, 0, 10]), ('485488', [1, 0, 0, 0, 5, 0, 0, 0, 0, 4, 2, 0, 3, 0, 0, 0, 3, 0, 0, 5, 0, 4, 5, 1, 0, 4, 0, 0, 0, 0, 2, 1])]


In [3]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

for i in xrange(len(trainRDD)):
    record = Record((trainRDD[i][0], trainRDD[i][1]))
    lsh.add(record)


In [4]:
shinglesText = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
test = (492346,shingles(shinglesText,SHINGLES_LEN))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
testRecord = Record(test)

In [5]:
print lsh.getAllSimilarRecords(testRecord)

[225529]
